In [1]:
import pandas as pd
import json
import os
import re
from datasets import load_dataset

In [2]:
train_data = [json.loads(lines) for lines in open("/kaggle/input/dataset/train.jsonl")]
val_data = [json.loads(lines) for lines in open("/kaggle/input/dataset/val.jsonl")]
test_data = [json.loads(lines) for lines in open("/kaggle/input/dataset/test.jsonl")]

train = pd.DataFrame(train_data)
val = pd.DataFrame(val_data)
test = pd.DataFrame(test_data)

In [3]:
train.head()

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,"[Wes Welker Wanted Dinner With Tom Brady, But ...",reddit,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[how about that morning we go throw?],"[[[3, 151], [3, 186]]]",[passage]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,[NASA sets date for full recovery of ozone hole],Twitter,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",[phrase]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,[This is what makes employees happy -- and it'...,Twitter,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",[phrase]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,844567852531286016,[Passion is overrated — 7 work habits you need...,Twitter,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",There's a lot more to work that loving your job,"business, work-life, careers",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[Purpose connects us to something bigger and i...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",[multi]
4,31b108a3-c828-421a-a4b9-cf651e9ac859,814186311573766144,[The perfect way to cook rice so that it's per...,Twitter,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[in a rice cooker],"[[[5, 60], [5, 76]]]",[phrase]


In [4]:
def preprocessing(text):
  if not isinstance(text, str):
    text = str(text)
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  return text

In [5]:
train['postText'] = train['postText'].apply(lambda x : [preprocessing(text) for text in x])
train['targetParagraphs'] = train['targetParagraphs'].apply(lambda x : [preprocessing(text) for text in x])
train['targetTitle'] = train['targetTitle'].apply(preprocessing)

train.head(4)

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,[wes welker wanted dinner with tom brady but p...,reddit,[itll be just like old times this weekend for ...,wes welker wanted dinner with tom brady but pa...,It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[how about that morning we go throw?],"[[[3, 151], [3, 186]]]",[passage]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,[nasa sets date for full recovery of ozone hole],Twitter,[2070 is shaping up to be a great year for mot...,hole in ozone layer expected to make full reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",[phrase]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,[this is what makes employees happy and its n...,Twitter,[despite common belief money isnt the key to e...,intellectual stimulation trumps money for empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",[phrase]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,844567852531286016,[passion is overrated 7 work habits you need ...,Twitter,[its common wisdom near gospel really and not ...,follow your passion is wrong here are 7 habits...,There's a lot more to work that loving your job,"business, work-life, careers",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[Purpose connects us to something bigger and i...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",[multi]


In [6]:
val['postText'] = val['postText'].apply(lambda x : [preprocessing(text) for text in x])
val['targetParagraphs'] = val['targetParagraphs'].apply(lambda x : [preprocessing(text) for text in x])
val['targetTitle'] = val['targetTitle'].apply(preprocessing)

val.head(4)

,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags,id
0,800048986762423_885081814925806,[five nights at freddys sequel delayed for wei...,Facebook,[five nights at freddys creator scott cawthon ...,five nights at freddys sequel delayed for weir...,Five Nights at Freddy's creator Scott Cawthon ...,None,[https://gamerant.com/wp-content/uploads/five-...,https://gamerant.com/five-nights-at-freddys-se...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[some of the plot elements are so disturbing t...,"[[[2, 158], [2, 236]]]",[passage],0
1,4jp20e,[why arizona sheriff joe arpaios fate could ha...,reddit,[ reuterslaura segall maricopa county sheriff ...,why arizona sheriff joe arpaios fate could han...,"<p>A single word — ""intentionally"" — could tra...",None,[https://archive.is/0eGBT/bbf6d79b366fc7e75b8b...,https://archive.is/0eGBT,"{'source': 'anonymized', 'humanSpoiler': '""Int...","[""intentionally"", could transform a court case...","[[[0, 197], [0, 212]], [[0, 215], [0, 328]]]",[multi],1
2,857559227384160256,[heres how much you should be tipping your hai...,Twitter,[heres how much you should be tipping your hai...,heres how much you should be tipping your hair...,Remembering how much you’re supposed to tip an...,,None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[20%],"[[[3, 58], [3, 61]]]",[phrase],2
3,389091583213453312,[harry potter alums reunite for new movie],Twitter,[the mythology of punk musics evolution can be...,alan rickman rupert grint on cbgb reuniting p...,The mythology of punk music's evolution can be...,"Alan Rickman,Hilly Kristal,new rupert grint mo...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1ccNwKJ,"{'source': 'anonymized', 'humanSpoiler': 'Alan...","[Alan Rickman & Rupert Grint, CBGB]","[[[-1, 0], [-1, 27]], [[0, 98], [0, 102]]]",[multi],3


In [7]:
test['postText'] = test['postText'].apply(lambda x : [preprocessing(text) for text in x])
test['targetParagraphs'] = test['targetParagraphs'].apply(lambda x : [preprocessing(text) for text in x])
test['targetTitle'] = test['targetTitle'].apply(preprocessing)

test.head(4)

,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,id
0,170016066732231_222926014774569,[he tackles a nurse at the hospital then you s...,Facebook,[when you think about your good friends many t...,male nurse breaks down when his friend reveals...,"When you think about your good friends, many t...",None,[https://cdn0.littlethings.com/app/uploads/201...,http://www.littlethings.com/surprise-gift-kidn...,0
1,828732026186563584,[why you should be selfish at work],Twitter,[were always being encouraged to help others b...,why you should be selfish at work helping othe...,Wharton School of the University of Pennsylvan...,"Why,selfish,work",None,None,1
2,721323075246112769,[the one strange trick that will make you live...,Twitter,[meditating inside a beautiful stockphoto room...,the one strange trick that will help you live ...,A new study showed that the wealthy are living...,"culture, news, lgbtq, politics, journalism, vi...",[https://vice-images.vice.com/images/content-i...,http://bit.ly/1SbRoUK,2
3,51a6xo,[nerd wins scrabble championship with word you...,reddit,"[image getty, as you probably already know the...",nerd wins scrabble championship with word youv...,"As you probably already know, the World Scrabb...","good vocabularies, scrabble, braconid, Gizmodo","[data:image/gif;base64,R0lGODlhAQABAAAAACH5BAE...",http://archive.is/0z4xs,3


In [8]:
final_train = pd.DataFrame(columns= ['text', 'labels'])
final_val = pd.DataFrame(columns= ['text', 'labels'])

In [9]:
def integrate_text_features(df):
    inputs = []

    for _, row in df.iterrows():
        input_text = row['postText']
        input_text.extend(row['targetParagraphs'])
        input_text.append(row['targetTitle'])
        inputs.append(input_text)
        
    return inputs

In [10]:
train_inputs = integrate_text_features(train)
val_inputs= integrate_text_features(val)

In [11]:
final_train['text'] = train_inputs
final_train['labels'] = train['tags']

In [12]:
final_val['text'] = val_inputs
final_val['labels'] = val['tags']

In [13]:
final_val['text'][0]

['five nights at freddys sequel delayed for weird reason',
 'five nights at freddys creator scott cawthon takes to steam to tease a possible delay for five nights at freddys sister location the fifth game in the series',
 'for the past couple of years horror gaming fans have been able to look forward to one new entry in the five nights at freddys series after another with four core games one rpg spinoff and a novel released so far the next game in the franchise five nights at freddys sister location was scheduled to release this coming friday october 7th but if developer scott cawthon is to be believed the project has been delayed by a few months',
 'according to a post by cawthon on the five nights at freddys sister location steam page the game is being delayed because its too dark cawthon said that some of the plot elements are so disturbing that they are making him feel sick and so he is thinking about delaying the game so that he can rework it entirely into something kidfriendly',


In [14]:
final_train.to_csv('train_data.csv', index=False, encoding='utf-8')

In [15]:
final_val.to_csv('val_data.csv', index = False, encoding= 'utf-8')

In [16]:
test_prep = pd.DataFrame(columns= ['id','features'])

In [17]:
test_features = integrate_text_features(test)

In [18]:
test_prep['id'] = test['id']
test_prep['text'] = test_features

In [19]:
test_prep.head()

,id,features,text
0,0,NaN,[he tackles a nurse at the hospital then you s...
1,1,NaN,"[why you should be selfish at work, were alway..."
2,2,NaN,[the one strange trick that will make you live...
3,3,NaN,[nerd wins scrabble championship with word you...
4,4,NaN,[the bizarre new way to eat eggs that has ever...


In [20]:
test_prep.to_csv('test.csv', index=False, encoding='utf-8')

In [21]:
train_df = pd.read_csv('/kaggle/working/train_data.csv')
train_df.head()

,text,labels
0,['wes welker wanted dinner with tom brady but ...,['passage']
1,['nasa sets date for full recovery of ozone ho...,['phrase']
2,['this is what makes employees happy and its ...,['phrase']
3,['passion is overrated \xa07 work habits you n...,['multi']
4,['the perfect way to cook rice so that its per...,['phrase']
